In [19]:
import awswrangler as wr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from tableone import TableOne

In [20]:
import os

In [21]:
os.getcwd()

'/home/ec2-user/SageMaker/mimic-iv-datathon'

In [136]:
patients= pd.read_csv("/home/ec2-user/SageMaker/initial_cohort_no_covariates.csv")

In [188]:
patients.head()

,subject_id,hadm_id,stay_id,charttime,creat_low_past_7day,creat_low_past_48hr,creat,aki_stage_creat,uo_rt_6hr,uo_rt_12hr,...,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,creat_24,urine_24,first_careunit
0,10002155,23822395,33685454,2129-08-05 05:01:00,0.9,0.9,1.2,1,NaN,NaN,...,2129-08-04 12:35:00,0,F,80,2128,2008 - 2010,2131-03-10,1,1,Coronary Care Unit (CCU)
1,10002155,23822395,33685454,2129-08-06 06:05:00,0.9,0.9,1.2,1,NaN,NaN,...,2129-08-04 12:35:00,0,F,80,2128,2008 - 2010,2131-03-10,1,1,Coronary Care Unit (CCU)
2,13774492,21066890,30073242,2125-05-20 03:01:00,0.3,0.3,0.5,1,NaN,NaN,...,2125-05-10 10:01:00,0,F,57,2123,2014 - 2016,NaN,1,1,Medical/Surgical Intensive Care Unit (MICU/SICU)
3,13774492,21066890,30073242,2125-05-21 04:36:00,0.3,0.5,0.5,1,NaN,NaN,...,2125-05-10 10:01:00,0,F,57,2123,2014 - 2016,NaN,1,1,Medical/Surgical Intensive Care Unit (MICU/SICU)
4,13774492,21066890,30073242,2125-05-19 05:46:00,0.3,0.3,0.5,1,NaN,NaN,...,2125-05-10 10:01:00,0,F,57,2123,2014 - 2016,NaN,1,1,Medical/Surgical Intensive Care Unit (MICU/SICU)


In [131]:
icd9codes = wr.athena.read_sql_query("SELECT * from diagnoses_icd", database="mimiciv")

In [132]:
icustay= wr.athena.read_sql_query("SELECT * from icustays", database="mimiciv")

In [137]:
# ADD ICU ADMISSION UNIT
patients = patients.join(icustay[["first_careunit", "stay_id"]].set_index("stay_id"), on=['stay_id'])

In [138]:
icd9codes.head(5)

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10002013,23581541,17,311,9
1,10002013,23581541,18,30000,9
2,10002013,23581541,19,412,9
3,10002013,23581541,20,V4582,9
4,10002013,23581541,21,V1582,9


In [139]:
# add comorbidities per stay = "stay_id"
icustay.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit',
       'intime', 'outtime', 'los'],
      dtype='object')

In [140]:
# merge with icustays
print("rows before adding icustay id: %s"%icd9codes.shape[0])
print("n° subjects: %s"%icd9codes.subject_id.nunique())

icd9codes_1 = icd9codes.join(icustay[['subject_id','hadm_id',"stay_id"]].set_index(['subject_id','hadm_id']), on=['subject_id','hadm_id'])

print("rows after adding icustay id %s"%icd9codes_1.shape[0])
print("n° subjects %s"%icd9codes_1.subject_id.nunique())


rows before adding icustay id: 5006884
n° subjects: 190179
rows after adding icustay id 5182587
n° subjects 190179


In [142]:
# FILTER POPULATIONS taking only those who are in the aki choort
icd9codes_1=icd9codes_1[icd9codes_1.stay_id.isin(patients.stay_id.unique().tolist())]


In [143]:
print("n° subjects after filtering population for the ones in AKI choort %s"%icd9codes_1.subject_id.nunique())

n° subjects after filtering population for the ones in AKI choort 12232


In [144]:
print("n° subjects %s"%icd9codes_1.subject_id.nunique())
print("n° stays %s"%icd9codes_1.stay_id.nunique())


n° subjects 12232
n° stays 13354


In [145]:
# extract comorbidities 
icd9codes_1.icd_version.value_counts()

9     174167
10    103938
Name: icd_version, dtype: Int64

In [146]:
icd9codes_1.sort_values(by="icd_version").head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,stay_id
193,10002155,23822395,1,41011,9,33685454
2974558,18227591,29010365,11,1974,9,31443543
2974558,18227591,29010365,11,1974,9,35277072
2974559,18227591,29010365,12,5373,9,31443543
2974559,18227591,29010365,12,5373,9,35277072


In [147]:
icd9codes_1[icd9codes_1.icd_version==9].stay_id.nunique()

8802

In [148]:
icd9codes_1[icd9codes_1.icd_version==10].stay_id.nunique()

4552

In [149]:
# DAIBETES

In [150]:
#icd 9
def is_DM(row):
    
    if row.icd_version== 9:
        if row.icd_code == 7751:
            return 1 
        if pd.to_numeric(row.icd_code, errors= "coerce") >= 25000 and pd.to_numeric(row.icd_code, errors= "coerce") <= 25033:
            return 1      
        if pd.to_numeric(row.icd_code, errors= "coerce") >= 64800 and pd.to_numeric(row.icd_code, errors= "coerce") <= 64804:
            return 1
        if pd.to_numeric(row.icd_code, errors= "coerce") >= 24900 and pd.to_numeric(row.icd_code, errors= "coerce") <= 24931:
            return 1      
        if pd.to_numeric(row.icd_code, errors= "coerce") >= 25040 and pd.to_numeric(row.icd_code, errors= "coerce") <= 25093:
            return 1
        if pd.to_numeric(row.icd_code, errors= "coerce") >= 24940 and pd.to_numeric(row.icd_code, errors= "coerce") <= 24991:
            return 1
        else:
            return 0 
        
    if row.icd_version== 10:       
        if row.icd_code.startswith('E08') or row.icd_code.startswith('E09') or row.icd_code.startswith('E10') or row.icd_code.startswith('E11') or row.icd_code.startswith('E12') or row.icd_code.startswith('E13'):
            return 1      
        else:
            return 0 
        
icd9codes_1['DM']=icd9codes_1.apply (lambda row:is_DM(row), axis=1)

In [151]:
icd9codes_1[icd9codes_1['DM']==1].DM.sum()

5057

In [152]:
print("n° of stays with DM")
icd9codes_1[icd9codes_1['DM']==1].groupby("stay_id").max().DM.sum()

n° of stays with DM


4263

In [153]:
icd9codes_1[icd9codes_1['DM']==1].groupby("stay_id").max().DM.sum()/icd9codes_1.stay_id.nunique()

0.3192301932005392

In [154]:
def is_HF(row): #heart failure
    if row.icd_version== 9:
        if pd.to_numeric(row.icd_code, errors= "coerce") == 4280 or pd.to_numeric(row.icd_code, errors= "coerce") == 4281 or pd.to_numeric(row.icd_code, errors= "coerce") == 4289 or pd.to_numeric(row.icd_code, errors= "coerce")==39891:
            return 1      
        if pd.to_numeric(row.icd_code, errors= "coerce") >= 42820 and pd.to_numeric(row.icd_code, errors= "coerce") <= 42823:
            return 1
        if pd.to_numeric(row.icd_code, errors= "coerce") >= 42830 and pd.to_numeric(row.icd_code, errors= "coerce") <= 42832:
            return 1      
        if pd.to_numeric(row.icd_code, errors= "coerce") >= 42840 and pd.to_numeric(row.icd_code, errors= "coerce") <= 42843:
            return 1
        else:
            return 0 
        
    if row.icd_version== 10:
        if row.icd_code.startswith('I13') or row.icd_code.startswith('I11') or row.icd_code.startswith('I25') or row.icd_code.startswith('I46') or row.icd_code.startswith('I50') or row.icd_code.startswith('I42') or row.icd_code.startswith('I43')  or row.icd_code.startswith('I44'):
            return 1      
        else:
            return 0         
        
        
icd9codes_1['Heart Failure']=icd9codes_1.apply (lambda row:is_HF(row), axis=1)

In [155]:
icd9codes_1[icd9codes_1['Heart Failure']==1]['Heart Failure'].sum()

9797

In [156]:
print("n° of stays with Heart Failure")
icd9codes_1[icd9codes_1['Heart Failure']==1].groupby("stay_id").max()['Heart Failure'].sum()

n° of stays with Heart Failure


5346

In [157]:
icd9codes_1[icd9codes_1['Heart Failure']==1].groupby("stay_id").max()['Heart Failure'].sum()/icd9codes_1.stay_id.nunique()

0.400329489291598

In [173]:
# Hypertension -icd 9 and 10 
def is_Hypert(row): 
    
    l=[4010,4011,4019,4372,40200,40210,40290,40509,40519,40201,40211,40291,40310,40300,40390,40501,40511,
       40591,40301,40311,40391,40400,40410,40490,40401,40411,40491,40402,40412,40492,40403,40413,40493]
    
    if row.icd_version== 9:

        if pd.to_numeric(row.icd_code, errors= "coerce") in l:
            return 1      
        if pd.to_numeric(row.icd_code, errors= "coerce") >= 64200 and pd.to_numeric(row.icd_code, errors= "coerce") <= 64204:
            return 1
        if pd.to_numeric(row.icd_code, errors= "coerce") >= 64220 and pd.to_numeric(row.icd_code, errors= "coerce") <= 64224:
            return 1      
        if pd.to_numeric(row.icd_code, errors= "coerce") >= 64210 and pd.to_numeric(row.icd_code, errors= "coerce") <= 64214:
            return 1
        if pd.to_numeric(row.icd_code, errors= "coerce") >= 64270 and pd.to_numeric(row.icd_code, errors= "coerce") <= 64274:
            return 1   
        if pd.to_numeric(row.icd_code, errors= "coerce") >= 64290 and pd.to_numeric(row.icd_code, errors= "coerce") <= 64294:
            return 1   
        else:
            return 0 
        
    if row.icd_version== 10:
        if row.icd_code.startswith('I10') or row.icd_code.startswith('I15') or row.icd_code.startswith('I16'):
            return 1      
        else:
            return 0  
icd9codes_1['Hypertension']=icd9codes_1.apply (lambda row:is_Hypert(row), axis=1)

In [174]:
icd9codes_1[icd9codes_1['Hypertension']==1]['Hypertension'].sum()

8105

In [176]:
print("n° of stays with Hypertension")
icd9codes_1[icd9codes_1['Hypertension']==1].groupby("stay_id").max()['Hypertension'].sum()

n° of stays with Hypertension


8038

In [177]:
icd9codes_1[icd9codes_1['Hypertension']==-1].head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,stay_id,DM,Heart Failure,Hypertension,CKD


In [178]:
icd9codes_1[icd9codes_1['Hypertension']==1].groupby("stay_id").max()['Hypertension'].sum()/icd9codes_1.stay_id.nunique()

0.6019170286056612

In [162]:
def is_CKD(row):
    if row.icd_version== 9:
        
        if pd.to_numeric(row.icd_code, errors= "coerce") >= 5851 and pd.to_numeric(row.icd_code, errors= "coerce") <= 5856:
            return 1     
        if pd.to_numeric(row.icd_code, errors= "coerce") == 5859:
            return 1   
        else:
            return 0 
        
    if row.icd_version== 10:
        if row.icd_code.startswith('I12') or row.icd_code.startswith('I12') or row.icd_code.startswith('N01') or row.icd_code.startswith('N02') or row.icd_code.startswith('N03') or row.icd_code.startswith('N04') or row.icd_code.startswith('N05') or row.icd_code.startswith('N06') or row.icd_code.startswith('N07') or row.icd_code.startswith('N08') or row.icd_code.startswith('N11') or row.icd_code.startswith('N13')     or row.icd_code.startswith('N14') or row.icd_code.startswith('N15') or row.icd_code.startswith('N18') or row.icd_code.startswith('N19') or row.icd_code.startswith('N25') or row.icd_code.startswith('N26') or row.icd_code.startswith('N27') or row.icd_code.startswith('N28') or row.icd_code.startswith('N29'):
            return 1      
        else:
            return 0  
icd9codes_1['CKD']=icd9codes_1.apply (lambda row:is_CKD(row), axis=1)

In [163]:
print("n° of stays with CKD")
print(icd9codes_1[icd9codes_1['CKD']==1].groupby("stay_id").max()['CKD'].sum())


n° of stays with CKD
2608


In [164]:
icd9codes_1[icd9codes_1['CKD']==1].groupby("stay_id").max()['CKD'].sum()/icd9codes_1.stay_id.nunique()

0.1952972892017373

In [183]:
ckd   = icd9codes_1.groupby('stay_id')['CKD'].max().reset_index()
dm    = icd9codes_1.groupby('stay_id')['DM'].max().reset_index()
hyper = icd9codes_1.groupby('stay_id')['Hypertension'].max().reset_index()
hf    = icd9codes_1.groupby('stay_id')['Heart Failure'].max().reset_index()

In [184]:
total_cmor= ckd.merge(dm, on=['stay_id'])
total_cmor.shape[0]

13354

In [185]:
total_cmor= total_cmor.merge(hyper, on=['stay_id'])
total_cmor.shape[0]

13354

In [186]:
total_cmor= total_cmor.merge(hf, on=['stay_id'])
total_cmor.shape[0]

13354

In [187]:
total_cmor.head(30)

,stay_id,CKD,DM,Hypertension,Heart Failure
0,30001396,1,0,1,0
1,30001446,0,0,0,0
2,30001555,0,0,0,0
3,30004018,0,0,1,0
4,30005366,1,1,1,1
5,30006565,0,0,0,0
6,30006874,1,0,1,0
7,30006983,0,0,0,0
8,30007175,0,0,0,1
9,30008785,1,1,0,0


In [190]:
total_cmor.to_csv("/home/ec2-user/SageMaker/covariates.csv")

In [189]:
patients.to_csv("/home/ec2-user/SageMaker/initial_cohort_no_covariates_icutypes.csv")